# Sedona Experiments

Read in data

In [2]:
from sedona.spark import SedonaContext
from pyspark.sql.functions import expr

# Start Spark + Sedona
config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    config("spark.sql.legacy.parquet.nanosAsLong", "true"). \
    getOrCreate()
sedona = SedonaContext.create(config)

25/05/26 22:32:35 WARN Utils: Your hostname, cs-spatial-501 resolves to a loopback address: 127.0.0.1; using 128.101.33.153 instead (on interface eno1)
25/05/26 22:32:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/uribe055/.ivy2/cache
The jars for the packages stored in: /home/uribe055/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
edu.ucar#cdm-core added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe87590f-98b7-4ece-8036-ca0788edf2e1;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/uribe055/sedona_experiments/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

In [3]:
sdf = sedona.read.parquet(f"/home/uribe055/sedona_experiments/processed_data/*.parquet")

25/05/26 22:33:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Create a geometry column for spatial queries

In [4]:
sdf = sdf.withColumn("geom", expr("ST_Point(longitude, latitude)")).drop("number", "expver")
sdf.createOrReplaceTempView("GL")
sdf.show()

+--------+---------+-------------------+--------------+-----------------+
|latitude|longitude|               time|2m_temperature|             geom|
+--------+---------+-------------------+--------------+-----------------+
|    84.0|    -75.0|2023-12-27 00:00:00|     243.76312|   POINT (-75 84)|
|    84.0|   -74.75|2023-12-27 00:00:00|     243.75336|POINT (-74.75 84)|
|    84.0|    -74.5|2023-12-27 00:00:00|     243.74554| POINT (-74.5 84)|
|    84.0|   -74.25|2023-12-27 00:00:00|     243.73773|POINT (-74.25 84)|
|    84.0|    -74.0|2023-12-27 00:00:00|     243.72015|   POINT (-74 84)|
|    84.0|   -73.75|2023-12-27 00:00:00|     243.66351|POINT (-73.75 84)|
|    84.0|    -73.5|2023-12-27 00:00:00|     243.60687| POINT (-73.5 84)|
|    84.0|   -73.25|2023-12-27 00:00:00|     243.55023|POINT (-73.25 84)|
|    84.0|    -73.0|2023-12-27 00:00:00|     243.49554|   POINT (-73 84)|
|    84.0|   -72.75|2023-12-27 00:00:00|      243.4389|POINT (-72.75 84)|
|    84.0|    -72.5|2023-12-27 00:00:0

### Functions

In [ ]:
# def daily_05(start_time, end_time, min_lat,max_lat,min_lon,max_lon):
#     query = f"""
#         SELECT time, int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)
#         FROM GL
#         WHERE latitude BETWEEN {min_lat} AND {max_lat}
#             AND longitude BETWEEN {min_lon} AND {max_lon}
#             AND time BETWEEN '{start_time}' AND '{end_time}'
#         GROUP BY time, int(latitude/0.5), int(longitude/0.5)
#         """
#     return query

# def daily_1(start_time, end_time, min_lat,max_lat,min_lon,max_lon):
#     query = f"""
#         SELECT time, int(latitude), int(longitude), avg(2m_temperature)
#         FROM GL
#         WHERE latitude BETWEEN {min_lat} AND {max_lat}
#             AND longitude BETWEEN {min_lon} AND {max_lon}
#             AND time BETWEEN '{start_time}' AND '{end_time}'
#         GROUP BY time, int(latitude), int(longitude)
#         """
#     return query

In [23]:
def daily(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT time, latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY time, latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT time, int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY time, int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT time, int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY time, int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [25]:
def monthly(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT month(time), latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY month(time), latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT month(time), int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY month(time), int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT month(time), int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY month(time), int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [28]:
def yearly(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT year(time), latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY year(time), latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT year(time), int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY year(time), int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT year(time), int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY year(time), int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [5]:
def get_heatmap_query(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY latitude, longitude"""
    else:   # 1.0
        q0 = f"""SELECT int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [50]:
def get_find_time_query(t, start_time, end_time, min_lat, max_lat, min_lon, max_lon,filter_value):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    if t == "month":
        q0=f"""SELECT month(time), avg(2m_temperature)"""
        q2=f"""GROUP BY month(time)
            HAVING avg(2m_temperature) > {filter_value}
            """
    elif t == "hour":
        q0=f"""SELECT time, avg(2m_temperature)"""
        q2=f"""GROUP BY time
            HAVING avg(2m_temperature) > {filter_value}
            """
    else:   # t == "year"
        q0=f"""SELECT year(time), avg(2m_temperature)"""
        q2=f"""GROUP BY year(time)
            HAVING avg(2m_temperature) > {filter_value}
            """

    return q0 + q1 + q2

In [ ]:
result = sedona.sql(yearly(1.0, "2020-01-01 00:00:00","2024-12-31 23:00:00",60,85,-69,-10,))
result.show()

---

### Changing Spatial and Temporal Resolution

In [38]:
import time
import pandas as pd
import os

t_res = ["hour","hour","hour", "day","day","day", "year","year","year", "month","month"]
s_res = [0.25, 0.5, 1.0, 0.25, 0.5, 1.0, 0.25, 0.5, 1.0, 0.5, 1.0]

start_time = "2020-01-01 00:00:00"
end_time = "2024-12-31 23:00:00"
min_lat = 60
max_lat = 80
min_lon = -60
max_lon = -30
outfilename = "/home/uribe055/sedona_experiments/results_" + "changing_resolution.csv"

results_list = []
for t, s in zip(t_res, s_res):
    if t == "hour":
        query = f"""SELECT time, latitude, longitude, avg(2m_temperature)
                FROM GL
                WHERE latitude BETWEEN {min_lat} AND {max_lat}
                    AND longitude BETWEEN {min_lon} AND {max_lon}
                    AND time BETWEEN '{start_time}' AND '{end_time}'
                GROUP BY time, latitude, longitude
                """
    elif t == "day":
        query = daily(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
    elif t == "month":
        query = monthly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
    else:
        query = yearly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)

    query_start = time.time()
    result = sedona.sql(query)
    result.collect()
    query_end = time.time()

    results_list.append({"sys": "Sedona", 
                         "t_res": t,
                         "s_res": s,
                         "time_span":5,             # FOR HEATMAP
                         "tr": None,
                         "ta": None,
                         "total_time": query_end - query_start,
                         "percent_area":100     # FOR CHANGING RESULT SIZE
                        })

results_df = pd.DataFrame(results_list)
out_file = os.path.join(outfilename)
results_df.to_csv(out_file, index=False)

### Changing Result Size

In [40]:
# res_pairs = [("hour", 0.25), ("year", 0.25), ("month", 0.5), ("hour", 1.0), ("year", 1.0)]

import time
import pandas as pd
import os


start_time = "2020-01-01 00:00:00"
end_time = "2024-12-31 23:00:00"
t_res = ["hour", "year", "month", "hour", "year"]
s_res = [0.25, 0.25, 0.5, 1.0, 1.0]

coord_dict = {"1": [83,86,-20,-15],
              "25": [65,80,-55,-30],
              "50": [60,85,-60,-30]
}
percent_area = [1, 25, 50]
outfilename = "/home/uribe055/sedona_experiments/results_" + "changing_result_size.csv"

results_list = []
for t, s in zip(t_res, s_res):
    for pa in percent_area:
        coords = coord_dict[str(pa)]
        min_lat = coords[0]
        max_lat = coords[1]
        min_lon = coords[2]
        max_lon = coords[3]
        if t == "hour":
            query = f"""SELECT time, latitude, longitude, avg(2m_temperature)
                    FROM GL
                    WHERE latitude BETWEEN {min_lat} AND {max_lat}
                        AND longitude BETWEEN {min_lon} AND {max_lon}
                        AND time BETWEEN '{start_time}' AND '{end_time}'
                    GROUP BY time, latitude, longitude
                    """
        elif t == "day":
            query = daily(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
        elif t == "month":
            query = monthly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
        else:
            query = yearly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)

        query_start = time.time()
        result = sedona.sql(query)
        result.collect()
        query_end = time.time()

        results_list.append({"sys": "Sedona", 
                            "t_res": t,
                            "s_res": s,
                            "time_span":5,             # FOR HEATMAP
                            "tr": None,
                            "ta": None,
                            "total_time": query_end - query_start,
                            "percent_area":pa     # FOR CHANGING RESULT SIZE
                            })

results_df = pd.DataFrame(results_list)
out_file = os.path.join(outfilename)
results_df.to_csv(out_file, index=False)

### Heatmap Query Performance

Ran three times to get better average of 2.5 year timespan point execution time.

In [6]:
# ["025_H", "025_Y", "05_M", "1_H", "1_Y"]

import time
import pandas as pd
import os

num_years_dict = {"1": ["2017-01-01 00:00:00", "2017-12-31 23:00:00"],
                  "2.5": ["2019-01-01 00:00:00", "2021-06-31 23:00:00"],
                  "5": ["2020-01-01 00:00:00", "2024-12-31 23:00:00"],
                  "10": ["2015-01-01 00:00:00", "2024-12-31 23:00:00"]
}
time_span = [1, 2.5, 5, 10]
# t_res = ["year", "year"]
s_res = [0.25, 1.0]
min_lat = 60
max_lat = 80
min_lon = -60
max_lon = -30
outfilename = "/home/uribe055/sedona_experiments/results/results_heatmap_10yrs.csv"

results_list = []
for s in s_res:
    for num_yrs in time_span:
        timestamps = num_years_dict[str(num_yrs)]
        start_time = timestamps[0]
        end_time = timestamps[1]

        query = get_heatmap_query(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)

        query_start = time.time()
        result = sedona.sql(query)
        result.collect()
        query_end = time.time()

        results_list.append({"sys": "Sedona", 
                            "t_res": "year",
                            "s_res": s,
                            "time_span":num_yrs,             # FOR HEATMAP
                            "tr": None,
                            "ta": None,
                            "total_time": query_end - query_start,
                            "percent_area":100    # FOR CHANGING RESULT SIZE
                            })

results_df = pd.DataFrame(results_list)
out_file = os.path.join(outfilename)
results_df.to_csv(out_file, index=False)

In [ ]:
sedona.stop()

### Find Time Query Performance

In [54]:
import time
import pandas as pd
import os

start_time = "2020-01-01 00:00:00"
end_time = "2024-12-31 23:00:00"
t_res = ["hour", "month", "year"]
s_res = [1, 0.5, 1]
filter_values = [205, 240, 275, 310]
min_lat = 60
max_lat = 80
min_lon = -60
max_lon = -30
outfilename = "/home/uribe055/sedona_experiments/results_find_time.csv"

results_list = []
for t, s in zip(t_res, s_res):
    for val in filter_values:

        query = get_find_time_query(t, start_time,end_time,min_lat,max_lat,min_lon,max_lon,val)

        query_start = time.time()
        result = sedona.sql(query)
        result.collect()
        query_end = time.time()

        results_list.append({"sys": "Sedona", 
                            "t_res": t,
                            "s_res": s,
                            "time_span":5,             # FOR HEATMAP
                            "tr": None,
                            "ta": None,
                            "total_time": query_end - query_start,
                            "percent_area":100,    # FOR CHANGING RESULT SIZE
                            "filter_value":val
                            })

results_df = pd.DataFrame(results_list)
out_file = os.path.join(outfilename)
results_df.to_csv(out_file, index=False)